In [ ]:
# rough skeleton for our main ftn

from tdmclient import ClientAsync, aw
from cv import *
from kalman import *
from motion_control import *
from utils import *
from path_planning import *
import time
# and other imports

# repeated timer to get data from thymio(velocity, acceleration, proximity sensor)
from threading import Timer
from collections import deque
class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

#TODO: what more data do we need?
def get_data():
    aw(node.wait_for_variables({"motor.left.speed","motor.right.speed", "acc"}))
    return [node["motor.left.speed"],
            node["motor.right.speed"],
            list(node["acc"])]

In [ ]:
#TODO: replace with functions from motion control
#I(Andres) wrote this ftn bc I don't know how to use motion control module
def simple_controller(x,y,theta, target_x, target_y):
    dx = target_x - x
    dy = target_y - y
    desired_theta = np.arctan2(dy, dx)
    dtheta = (desired_theta - theta + np.pi)% (2*np.pi) - np.pi
    if abs(dtheta) > 5*np.pi / 180: # abs(dtheta) larger than 3 degree
        v_ = 0
        w_ = np.sign(dtheta)*10*np.pi/180
    else:
        v_ = 1.5
        w_ = 0
    return v_, w_

In [17]:
#connect to thymio
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 310bffb3-c918-41d5-957e-fe37de1934e6

In [ ]:
#main loop
#set parameters here
dt = 0.1
epsilon = 0.7 # to check if thymio is close enough to next target

# TODO:connect to camera

# TODO:initialize map

# TODO:compute global path
# global path should be given as a deque of waypoints + goal, with goal being the last
# in case of kidnapping, we call for a new global path. No need to re-initialize camera, map, ekf
global_path = deque()
global_path.append([0,10])
global_path.append([10,10])
global_path.append([10,0]) 

# initialize ekf with information from cv
ekf = ExtendedKalmanFilter(5,5,2,dt)
camera_blocked = True
initial_state = np.array([0,0,np.pi/2,0,0])
ekf.initialize_X(initial_state)

# main loop
next_target = global_path.popleft()
print(f"next_target = {next_target[0]}, {next_target[1]}")
target_reached = False
counter = 0
start_time = time.time()
while (True):
    #TODO: get data from camera(thymio_found, x, y, theta)
    thymio_found, x_camera, y_camera, theta_camera = False, 0,0,0
    camera_blocked = not thymio_found

    ekf.switch_mode(camera_blocked)

    #TODO: modify get_data() to have other necessary sensor information(whatever for local navigation)
    # for now it only returns left/right motor speed, acc
    data = get_data()
    ul = data[0]
    ur = data[1]
    acc_z = data[2][2]
    v,w = from_u_to_vw(ul, ur)
    u = np.array([v,w])
    z = np.array([x_camera, y_camera, theta_camera, v, w])

    #if kidnapping detected(from acceleration), stop for 3 sec, compute global path again, continue;
    if abs(acc_z - 22) > 3: # sudden change in acc_z indicates kidnapping
        print("kidnapping detected")
        node.send_set_variables(motors(0,0))
        await client.sleep(3)
        #TODO: compute new global path
        # global_path = compute_global_path()
        # next_target = global_path.popleft()
        continue

    # updating thymio state, obviously
    ekf.predict_and_update(u,z)
    x,y,theta,v,w = ekf.get_X()
    if counter % 15 == 0:
        print(f"state = {x:.2f}, {y:.2f}, {theta:.2f}, {v:.2f}, {w:.2f}")
    
    #  check if next_target is reached
    target_reached = np.linalg.norm(np.array([x - next_target[0], y - next_target[1]])) < epsilon
    if target_reached:
        if len(global_path)==0:
            print("goal reached, terminating")
            break
        else:
            print("heading towards next waypoint")
            next_target = global_path.popleft()
            print(f"next_target = {next_target[0]}, {next_target[1]}")

    #TODO: replace with proper motion control(global + local)
    #in the end, should return desired v and w
    v,w = simple_controller(x,y,theta, next_target[0], next_target[1])
    
    # use utils.py!
    ul, ur = from_vw_to_u(v,w)
    node.send_set_variables(motors(ul, ur))
    await client.sleep(dt)
    counter += 1

    # whatever happens, stop after 120 sec
    elapsed_time = time.time() - start_time
    if elapsed_time > 120:
        break

# stop thymio
node.send_set_variables(motors(0,0))

next_target = 0, 10
state = -0.00, -0.00, 1.57, 0.60, 0.13
state = -0.03, 1.87, 1.60, 1.42, 0.00
state = -0.09, 4.02, 1.59, 1.32, 0.00
state = -0.17, 6.13, 1.62, 1.21, -0.04
state = -0.17, 7.79, 1.54, 1.87, -0.09
heading towards next waypoint
next_target = 10, 10
state = -0.02, 10.14, 1.49, -0.11, -0.07
state = -0.03, 10.11, 1.27, -0.02, -0.16
state = -0.03, 10.09, 1.02, -0.00, -0.16
state = -0.06, 10.06, 0.78, -0.00, -0.16
state = -0.07, 10.05, 0.53, -0.00, -0.16
state = -0.09, 10.04, 0.29, 0.02, -0.17
state = -0.13, 10.04, 0.05, 0.05, -0.17
state = 1.57, 10.05, 0.04, 1.50, 0.01
state = 3.81, 10.11, 0.02, 1.39, 0.01
state = 6.02, 10.15, 0.01, 1.47, -0.01
state = 8.21, 10.18, 0.02, 0.03, -0.01
state = 9.11, 10.09, -0.11, 1.43, 0.01
heading towards next waypoint
next_target = 10, 0
state = 10.03, 9.97, -0.30, -0.02, -0.16
state = 10.01, 9.97, -0.54, 0.03, -0.16
state = 10.03, 9.96, -0.80, -0.07, -0.16
state = 10.01, 9.99, -1.05, -0.18, -0.13
state = 9.99, 10.01, -1.29, 0.00, -0.16
state

In [ ]:
# to unlock thymio...
await node.unlock()